# Memory Management - Part 1 - Sessions

In this notebook, you'll learn:

- ✅ What sessions are and how to use them in your agent
- ✅ How to build *stateful* agents with sessions and events
- ✅ How to persist sessions in a database
- ✅ Context management practices such as context compaction
- ✅ Best practices for sharing session State

---
# 1. Setup

In [2]:
import os
from dotenv import load_dotenv 

load_dotenv()
try:
    GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

from typing import Any, Dict

from google.adk.agents import Agent, LlmAgent
from google.adk.apps.app import App, EventsCompactionConfig
from google.adk.models.google_llm import Gemini
from google.adk.sessions import DatabaseSessionService
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.adk.tools.tool_context import ToolContext
from google.genai import types

print("✅ ADK components imported successfully.")

retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

✅ Gemini API key setup complete.
✅ ADK components imported successfully.


### Helper functions

Helper function that manages a complete conversation session, handling session
creation/retrieval, query processing, and response streaming. It supports
both single queries and multiple queries in sequence.

Example:

```
>>> await run_session(runner, "What is the capital of France?", "geography-session")
>>> await run_session(runner, ["Hello!", "What's my name?"], "user-intro-session")
```

In [ ]:
# Define helper functions that will be reused throughout the notebook
from typing import Union

async def run_session(
    runner_instance: Runner,
    # XZ: pass explicitly the two arguments
    #--------------------------------------
    session_service: Union[DatabaseSessionService, InMemorySessionService],
    user_id: str,
    model_name: str,
    # -------------------------------------
    user_queries: list[str] | str = None,
    session_id: str = "default",

):
    print(f"\n ### Session: {session_id}")

    # Get app name from the Runner
    app_name = runner_instance.app_name

    # Attempt to create a new session or retrieve an existing one
    try:
        session = await session_service.create_session(
            app_name=app_name, user_id=user_id, session_id=session_id
        )
    except:
        session = await session_service.get_session(
            app_name=app_name, user_id=user_id, session_id=session_id
        )

    # Process queries if provided
    if user_queries:
        # Convert single query to list for uniform processing
        if isinstance(user_queries,str):
            user_queries = [user_queries]

        # Process each query in the list sequentially
        for query in user_queries:
            print('-'*100)
            print(f"\nUser > {query}")

            # Convert the query string to the ADK Content format
            message = types.Content(role="user", parts=[types.Part(text=query)])

            # Stream the agent's response asynchronously
            async for event in runner_instance.run_async(
                user_id=user_id, session_id=session.id, new_message=message
            ):
                # Check if the event contains valid content
                if event.content and event.content.parts:
                    # Filter out empty or "None" responses before printing
                    if (
                        event.content.parts[0].text != "None"
                        and event.content.parts[0].text
                    ):
                        print(f"{model_name} > ", event.content.parts[0].text)
    else:
        print("No queries!")

print("✅ Helper functions defined.")

✅ Helper functions defined.


---
# 2. Session Management

**The Problem**

LLMs are stateless—they know only what you provide in a single API call. Without context management, every prompt starts from scratch.
ADK uses Sessions for short-term memory and Memory for long-term. This notebook covers Sessions.

**What is a Session**

A Session stores one continuous conversation: its history, tool calls, and responses. Each session belongs to one user and one agent.

Two components:
- Events: the conversation's building blocks—user inputs, agent responses, tool calls, tool outputs.
- State: a {key: value} scratchpad accessible to all subagents and tools.


**Managing Sessions**
- SessionService: stores and retrieves sessions (implementations vary: in-memory, database, cloud).
- Runner: orchestrates the conversation flow and maintains history automatically.

### Implementing First Stateful Agent

Lets start with a simple Session Management option (`InMemorySessionService`):

In [4]:
APP_NAME = "default"  # Application
USER_ID = "default"  # User
SESSION = "default"  # Session

MODEL_NAME = "gemini-2.5-flash-lite"


# Step 1: Create the LLM Agent
root_agent = Agent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="text_chat_bot",
    description="A text chatbot",  # Description of the agent's purpose
)

# Step 2: Set up Session Management
# InMemorySessionService stores conversations in RAM (temporary)
session_service = InMemorySessionService()

# Step 3: Create the Runner
runner = Runner(agent=root_agent, app_name=APP_NAME, session_service=session_service)

print("✅ Stateful agent initialized!")
print(f"   - Application: {APP_NAME}")
print(f"   - User: {USER_ID}")
print(f"   - Using: {session_service.__class__.__name__}")

✅ Stateful agent initialized!
   - Application: default
   - User: default
   - Using: InMemorySessionService


In [5]:
# Run a conversation with two queries in the same session
# Context is maintained
await run_session(
    runner,
    session_service,
    USER_ID,
    MODEL_NAME,
    [
        "Hello! who is P li",
        "Hello, remember that p li is a labradoodle who is the cutest of all, and she has a chinese name 虎俊生",
        "虎俊生 who"
    ],
    "first_p_li_session",
)


 ### Session: first_p_li_session
----------------------------------------------------------------------------------------------------

User > Hello! who is P li
gemini-2.5-flash-lite >  I'm sorry, but I don't have enough information to identify "P li". Could you please provide more context or details? For example, who is this person to you, or in what context have you heard this name?
----------------------------------------------------------------------------------------------------

User > Hello, remember that p li is a labradoodle who is the cutest of all, and she has a chinese name 虎俊生
gemini-2.5-flash-lite >  Thank you for the clarification! It's lovely to know that P Li is your adorable Labradoodle. "虎俊生" (Hǔ Jùn Shēng) is a wonderful Chinese name for her!

"虎" (Hǔ) means "tiger," which could signify strength, bravery, or a lively spirit – all fitting for a playful dog! "俊" (Jùn) means handsome, talented, or outstanding, and "生" (Shēng) means born or life. Together, it's a very 

But there's a catch: `InMemorySessionService` is temporary. **Once the application stops, all conversation history is lost.** 


In [6]:
# # Run this cell after restarting the kernel WITHOUT running the previous . All this history will be gone...
# await run_session(
#     runner,
#     session_service,
#     USER_ID,
#     MODEL_NAME,
#     [
#         "Hello! what did we talk about",
#     ],
#     "first_p_li_session",
# )

---
# 3. Persistent Sessions with `DatabaseSessionService`

While `InMemorySessionService` is great for prototyping, real-world applications need conversations to survive restarts, crashes, and deployments. Let's level up to persistent storage!

### Choosing the Right SessionService*

ADK provides different SessionService implementations for different needs:

| Service | Use Case | Persistence | Best For |
|---------|----------|-------------|----------|
| **InMemorySessionService** | Development & Testing | ❌ Lost on restart | Quick prototypes |
| **DatabaseSessionService** | Self-managed apps | ✅ Survives restarts | Small to medium apps |
| **Agent Engine Sessions** | Production on GCP | ✅ Fully managed | Enterprise scale |


Let's upgrade to one persistent session service type: `DatabaseSessionService` using SQLite.

In [7]:
# Step 1: Create the same agent (notice we use LlmAgent this time)
chatbot_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="text_chat_bot",
    description="A text chatbot with persistent memory",
)

# Step 2: Switch to DatabaseSessionService
# SQLite database will be created automatically
# !!! modified here
db_url = "sqlite+aiosqlite:///my_agent_data.db" # "sqlite:///my_agent_data.db"  # Local SQLite file
session_service = DatabaseSessionService(db_url=db_url)

# Step 3: Create a new runner with persistent storage
runner = Runner(agent=chatbot_agent, app_name=APP_NAME, session_service=session_service)

print("✅ Upgraded to persistent sessions!")
print(f"   - Database: my_agent_data.db")
print(f"   - Sessions will survive restarts!")

✅ Upgraded to persistent sessions!
   - Database: my_agent_data.db
   - Sessions will survive restarts!


In [8]:
await run_session(
    runner,
    session_service,
    USER_ID, 
    MODEL_NAME,
    [   "Hello, remember that p li is a labradoodle who is the cutest of all, and she has a chinese name 虎俊生",
        "虎俊生 who"],
    "first_p_li_session_with_db",
)


 ### Session: first_p_li_session_with_db
----------------------------------------------------------------------------------------------------

User > Hello, remember that p li is a labradoodle who is the cutest of all, and she has a chinese name 虎俊生
gemini-2.5-flash-lite >  Hello! I will remember that Pli is the cutest labradoodle ever and her Chinese name is 虎俊生.
----------------------------------------------------------------------------------------------------

User > 虎俊生 who
gemini-2.5-flash-lite >  虎俊生 is Pli, the cutest labradoodle!


In [9]:
# Run this cell after restarting the kernel WITHOUT running the previous . All this history is persistent
await run_session(
    runner,
    session_service,
    USER_ID, 
    MODEL_NAME,
    [   "what did i tell you about p li"],
    "first_p_li_session_with_db",
)


 ### Session: first_p_li_session_with_db
----------------------------------------------------------------------------------------------------

User > what did i tell you about p li
gemini-2.5-flash-lite >  You told me that Pli is a labradoodle who is the cutest of all, and her Chinese name is 虎俊生.


In [10]:
# verify that the session data is isolated BY creating a new session
await run_session(
    runner, session_service, USER_ID, MODEL_NAME, ["Hello! Who is p li?"], "a_random_session"
) 


 ### Session: a_random_session
----------------------------------------------------------------------------------------------------

User > Hello! Who is p li?
gemini-2.5-flash-lite >  I'm sorry, but I don't have any information about a person named "p li." Could you please provide more context or spell the name differently? I might be able to help if I have more details.


In [11]:
# check how data (events) is stored
import sqlite3

def check_data_in_db():
    with sqlite3.connect("my_agent_data.db") as connection:
        cursor = connection.cursor()
        result = cursor.execute(
            "select app_name, session_id, timestamp, event_data from events"
        )

        cols_list = [_[0] for _ in result.description]
        print(cols_list)

        rows = []
        for each in result.fetchall():
            rows.append(each)

        return cols_list, rows


cols, rows = check_data_in_db()

import pandas as pd 
pd.set_option('display.max_colwidth', None)
pd.DataFrame(rows, columns=cols)

['app_name', 'session_id', 'timestamp', 'event_data']


,app_name,session_id,timestamp,event_data
0,default,first_p_li_session_with_db,2026-01-31 21:19:22.159724,"{""content"": {""parts"": [{""text"": ""Hello, remember that p li is a labradoodle who is the cutest of all, and she has a chinese name \u864e\u4fca\u751f""}], ""role"": ""user""}, ""invocation_id"": ""e-87671840-368e-426c-8673-ab3d128171cd"", ""author"": ""user"", ""actions"": {""state_delta"": {}, ""artifact_delta"": {}, ""requested_auth_configs"": {}, ""requested_tool_confirmations"": {}}, ""id"": ""8e1440c6-65db-4736-baee-ba8e888019ec"", ""timestamp"": 1769923162.159724}"
1,default,first_p_li_session_with_db,2026-01-31 21:19:22.163414,"{""model_version"": ""gemini-2.5-flash-lite"", ""content"": {""parts"": [{""text"": ""Hello! I will remember that Pli is the cutest labradoodle ever and her Chinese name is \u864e\u4fca\u751f.""}], ""role"": ""model""}, ""finish_reason"": ""STOP"", ""usage_metadata"": {""candidates_token_count"": 25, ""prompt_token_count"": 59, ""prompt_tokens_details"": [{""modality"": ""TEXT"", ""token_count"": 59}], ""total_token_count"": 84}, ""invocation_id"": ""e-87671840-368e-426c-8673-ab3d128171cd"", ""author"": ""text_chat_bot"", ""actions"": {""state_delta"": {}, ""artifact_delta"": {}, ""requested_auth_configs"": {}, ""requested_tool_confirmations"": {}}, ""id"": ""360e988e-b3d2-45e4-970d-1869e5a00cda"", ""timestamp"": 1769923162.163414}"
2,default,first_p_li_session_with_db,2026-01-31 21:19:22.764288,"{""content"": {""parts"": [{""text"": ""\u864e\u4fca\u751f who""}], ""role"": ""user""}, ""invocation_id"": ""e-ca4f92e6-08ed-4b64-a897-81c901338752"", ""author"": ""user"", ""actions"": {""state_delta"": {}, ""artifact_delta"": {}, ""requested_auth_configs"": {}, ""requested_tool_confirmations"": {}}, ""id"": ""b0d98f52-7a4e-44b2-aeb4-1b492d9c80fe"", ""timestamp"": 1769923162.764288}"
3,default,first_p_li_session_with_db,2026-01-31 21:19:22.767464,"{""model_version"": ""gemini-2.5-flash-lite"", ""content"": {""parts"": [{""text"": ""\u864e\u4fca\u751f is Pli, the cutest labradoodle!""}], ""role"": ""model""}, ""finish_reason"": ""STOP"", ""usage_metadata"": {""candidates_token_count"": 13, ""prompt_token_count"": 90, ""prompt_tokens_details"": [{""modality"": ""TEXT"", ""token_count"": 90}], ""total_token_count"": 103}, ""invocation_id"": ""e-ca4f92e6-08ed-4b64-a897-81c901338752"", ""author"": ""text_chat_bot"", ""actions"": {""state_delta"": {}, ""artifact_delta"": {}, ""requested_auth_configs"": {}, ""requested_tool_confirmations"": {}}, ""id"": ""f244fb26-ffa9-4a70-bf0e-de2824eadca8"", ""timestamp"": 1769923162.767464}"
4,default,first_p_li_session_with_db,2026-01-31 21:19:23.260677,"{""content"": {""parts"": [{""text"": ""what did i tell you about p li""}], ""role"": ""user""}, ""invocation_id"": ""e-443eb76a-60ca-4c86-8ee4-333cb946a930"", ""author"": ""user"", ""actions"": {""state_delta"": {}, ""artifact_delta"": {}, ""requested_auth_configs"": {}, ""requested_tool_confirmations"": {}}, ""id"": ""81380b48-46c0-4571-93c9-448b97348710"", ""timestamp"": 1769923163.260677}"
5,default,first_p_li_session_with_db,2026-01-31 21:19:23.264256,"{""model_version"": ""gemini-2.5-flash-lite"", ""content"": {""parts"": [{""text"": ""You told me that Pli is a labradoodle who is the cutest of all, and her Chinese name is \u864e\u4fca\u751f.""}], ""role"": ""model""}, ""finish_reason"": ""STOP"", ""usage_metadata"": {""candidates_token_count"": 28, ""prompt_token_count"": 113, ""prompt_tokens_details"": [{""modality"": ""TEXT"", ""token_count"": 113}], ""total_token_count"": 141}, ""invocation_id"": ""e-443eb76a-60ca-4c86-8ee4-333cb946a930"", ""author"": ""text_chat_bot"", ""actions"": {""state_delta"": {}, ""artifact_delta"": {}, ""requested_auth_configs"": {}, ""requested_tool_confirmations"": {}}, ""id"": ""ab5ba012-c1ba-485f-9688-670ac5351e8a"", ""timestamp"": 1769923163.264256}"
6,default,a_random_session,2026-01-31 21:19:23.763644,"{""content"": {""parts"": [{""text"": ""Hello! Who is p li?""}], ""r

---
# 4. Context Compaction
> Problem: Too much data in database as time goes by

> Solution: Context compaction


### 4.1 Create an App for the agent


- give the app a name and pass in `chatbot_agent`. 
- create and pass in `EventsCompactionConfig` with two key variables:
    - **compaction_interval**: Asks the Runner to compact the history after every `n` conversations
    - **overlap_size**: Defines the number of previous conversations to retain for overlap



In [12]:
# Re-define our app with Events Compaction enabled
app_compacting = App(
    name="app_compacting",
    root_agent=chatbot_agent,
    # This is the new part!
    events_compaction_config=EventsCompactionConfig(
        compaction_interval=3,  # Trigger compaction every 3 invocations
        overlap_size=1,  # Keep 1 previous turn for context
    ),
)

db_url = "sqlite+aiosqlite:///my_agent_data.db"  # Local SQLite file
session_service = DatabaseSessionService(db_url=db_url)

# Create a new runner for our upgraded app
runner_compacting = Runner(
    app=app_compacting, session_service=session_service
)


print("✅ App upgraded with Events Compaction!")

✅ App upgraded with Events Compaction!


/var/folders/c1/gdb4b0xx4vd70kccs_c2tq3c0000gn/T/ipykernel_49067/1867897166.py:6: UserWarning: [EXPERIMENTAL] EventsCompactionConfig: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  events_compaction_config=EventsCompactionConfig(


### 4.2 Running the Demo

let's have a conversation that is long enough to trigger the compaction. 
- When you run the cell below, the output will look like a normal conversation. 
- However, because we configured our `App`, a compaction process will run silently in the background after the 3rd invocation.

In [13]:
session_id = "compaction_demo"
common_kargs = {'runner_instance': runner_compacting, 
                'session_service': session_service, 
                'user_id': USER_ID, 
                'model_name':MODEL_NAME,
                'session_id':session_id}

await run_session(
    user_queries = ["who is p li", 
                    "p li is my four year old daughter, who is a labradoodle",
                    "p li is 4 years old",
                    "p li licks",
                    "p li stinks",
                    "p li timid"],
    **common_kargs
)


 ### Session: compaction_demo
----------------------------------------------------------------------------------------------------

User > who is p li
gemini-2.5-flash-lite >  I'm sorry, but I don't have enough information to identify "p li". It's a very common combination of a single letter and a common surname.

To help me understand who you're referring to, could you please provide more details? For example:

*   **What is their full name?** (e.g., P. Li, Peter Li, Ping Li)
*   **What are they known for?** (e.g., a politician, an actor, a scientist, a character in a book/movie)
*   **What context are you thinking of?** (e.g., a news article, a historical event, a fictional story)

With more information, I might be able to help you identify them.
----------------------------------------------------------------------------------------------------

User > p li is my four year old daughter, who is a labradoodle
gemini-2.5-flash-lite >  Ah, thank you for clarifying!

"P Li" being your f

### 4.3 Verifying Compaction in the Session History


How: inspect the `events` list from our session. 

The compaction process **doesn't delete old events; it replaces them with a single, new `Event` that contains the summary.**

In [14]:
# Get the final session state
final_session = await session_service.get_session(
    app_name=runner_compacting.app_name,
    user_id=USER_ID,
    session_id=session_id,
)

print("--- Searching for Compaction Summary Event ---")
found_summary = False
for i, event in enumerate(final_session.events):
    # Compaction events have a 'compaction' attribute
    if event.actions and event.actions.compaction:
        print(f"✅ Event {i} is compaction event by {event.author}")
        print(f"\n Compacted information: {event}")
        found_summary = True

if not found_summary:
    print(
        "\n❌ No compaction event found. Try increasing the number of turns in the demo."
    )

--- Searching for Compaction Summary Event ---
✅ Event 6 is compaction event by user

 Compacted information: model_version=None content=None grounding_metadata=None partial=None turn_complete=None finish_reason=None error_code=None error_message=None interrupted=None custom_metadata=None usage_metadata=None live_session_resumption_update=None input_transcription=None output_transcription=None avg_logprobs=None logprobs_result=None cache_metadata=None citation_metadata=None interaction_id=None invocation_id='0fd32b7f-0f3a-4aed-9251-bc8598e0a603' author='user' actions=EventActions(skip_summarization=None, state_delta={}, artifact_delta={}, transfer_to_agent=None, escalate=None, requested_auth_configs={}, requested_tool_confirmations={}, compaction=EventCompaction(start_timestamp=1769923164.783505, end_timestamp=1769923166.83673, compacted_content=Content(
  parts=[
    Part(
      text='The user initially asked for information about "p li" and the AI was unable to identify anyone with t

In [15]:
# one row for each compacting event
cols, rows = check_data_in_db()

import pandas as pd 
pd.set_option('display.max_colwidth', None)
pd.DataFrame(rows, columns=cols).query("session_id == 'compaction_demo'")

['app_name', 'session_id', 'timestamp', 'event_data']


,app_name,session_id,timestamp,event_data
8,app_compacting,compaction_demo,2026-01-31 21:19:24.783505,"{""content"": {""parts"": [{""text"": ""who is p li""}], ""role"": ""user""}, ""invocation_id"": ""e-a5abafc9-f576-46f5-9810-2b1a7ac2483b"", ""author"": ""user"", ""actions"": {""state_delta"": {}, ""artifact_delta"": {}, ""requested_auth_configs"": {}, ""requested_tool_confirmations"": {}}, ""id"": ""eea12b84-2619-44b4-b4b4-26f7702c74ea"", ""timestamp"": 1769923164.783505}"
9,app_compacting,compaction_demo,2026-01-31 21:19:24.786493,"{""model_version"": ""gemini-2.5-flash-lite"", ""content"": {""parts"": [{""text"": ""I'm sorry, but I don't have enough information to identify \""p li\"". It's a very common combination of a single letter and a common surname.\n\nTo help me understand who you're referring to, could you please provide more details? For example:\n\n* **What is their full name?** (e.g., P. Li, Peter Li, Ping Li)\n* **What are they known for?** (e.g., a politician, an actor, a scientist, a character in a book/movie)\n* **What context are you thinking of?** (e.g., a news article, a historical event, a fictional story)\n\nWith more information, I might be able to help you identify them.""}], ""role"": ""model""}, ""finish_reason"": ""STOP"", ""usage_metadata"": {""candidates_token_count"": 160, ""prompt_token_count"": 35, ""prompt_tokens_details"": [{""modality"": ""TEXT"", ""token_count"": 35}], ""total_token_count"": 195}, ""invocation_id"": ""e-a5abafc9-f576-46f5-9810-2b1a7ac2483b"", ""author"": ""text_chat_bot"", ""actions"": {""state_delta"": {}, ""artifact_delta"": {}, ""requested_auth_configs"": {}, ""requested_tool_confirmations"": {}}, ""id"": ""baf3dc77-9110-41df-b3d7-536e2ff57621"", ""timestamp"": 1769923164.786493}"
10,app_compacting,compaction_demo,2026-01-31 21:19:25.814962,"{""content"": {""parts"": [{""text"": ""p li is my four year old daughter, who is a labradoodle""}], ""role"": ""user""}, ""invocation_id"": ""e-93f7a39b-4b33-4fdc-9c68-a29382d9c990"", ""author"": ""user"", ""actions"": {""state_delta"": {}, ""artifact_delta"": {}, ""requested_auth_configs"": {}, ""requested_tool_confirmations"": {}}, ""id"": ""a034810d-3022-4955-8b7c-fe214321a8b3"", ""timestamp"": 1769923165.814962}"
11,app_compacting,compaction_demo,2026-01-31 21:19:25.819177,"{""model_version"": ""gemini-2.5-flash-lite"", ""content"": {""parts"": [{""text"": ""Ah, thank you for clarifying!\n\n\""P Li\"" being your four-year-old daughter, who is a Labradoodle, paints a completely different and much more adorable picture!\n\nIn this context, \""P Li\"" is your beloved pet! Labradoodles are known for being intelligent, friendly, and energetic dogs, often with hypoallergenic coats. A four-year-old Labradoodle is likely in the prime of her life \u2013 playful, happy, and a wonderful companion.\n\nIt's lovely that you have such a special furry family member! Do you have any particular questions about P Li or Labradoodles in general?""}], ""role"": ""model""}, ""finish_reason"": ""STOP"", ""usage_metadata"": {""candidates_token_count"": 130, ""prompt_token_count"": 212, ""prompt_tokens_details"": [{""modality"": ""TEXT"", ""token_count"": 212}], ""total_token_count"": 342}, ""invocation_id"": ""e-93f7a39b-4b33-4fdc-9c68-a29382d9c990"", ""author"": ""text_chat_bot"", ""actions"": {""state_delta"": {}, ""artifact_delta"": {}, ""requested_auth_configs"": {}, ""requested_tool_confirmations"": {}}, ""id"": ""76ae50d6-444a-4b92-9e8f-1a13af84b97c"", ""timestamp"": 1769923165.819177}"
12,app_compacting,compaction_demo,2026-01-31 21:19:26.832795,"{""content"": {""parts"": [{""text"": ""p li is 4 years old""}], ""role"": ""user""}, ""invocation_id"": ""e-9c232ee5-ff0e-4000-a617-499326bc1cfa"", ""author"": ""user"", ""actions"": {""state_delta"": {}, ""artifact_delta"": {}, ""requested_auth_configs"": {}, ""requested_tool_confirmations"": {}}, ""id"": ""953f021c-9577-495c-8ff9-84889f57b38a"", ""timestamp"": 1769923166.832795}"
13,app_compacting,c

---
# 5.  Working with Session State

> Before: info isolated within each session

> Now: Share info across sessions

### 5.1 Creating custom tools for Session state management

Let's explore how to manually manage session state through custom tools. In this example, we'll identify a **transferable characteristic**, like a user's name and their country, and create tools to capture and save it.

**Why This Example?**

The username is a perfect example of information that:

- Is introduced once but referenced multiple times
- Should persist throughout a conversation
- Represents a user-specific characteristic that enhances personalization

Here, for demo purposes, we'll create two tools that can store and retrieve user name and country from the Session State. **Note that all tools have access to the `ToolContext` object.** You don't have to create separate tools for each piece of information you want to share. 

In [16]:
# Define scope levels for state keys (following best practices)
USER_NAME_SCOPE_LEVELS = ("temp", "user", "app")


# This demonstrates how tools can write to session state using tool_context.
# The 'user:' prefix indicates this is user-specific data.
def save_userinfo(
    tool_context: ToolContext, user_name: str, country: str
) -> Dict[str, Any]:
    """
    Tool to record and save user name and country in session state.

    Args:
        user_name: The username to store in session state
        country: The name of the user's country
    """
    # Write to session state using the 'user:' prefix for user data
    tool_context.state["user:name"] = user_name
    tool_context.state["user:country"] = country

    return {"status": "success"}


# This demonstrates how tools can read from session state.
def retrieve_userinfo(tool_context: ToolContext) -> Dict[str, Any]:
    """
    Tool to retrieve user name and country from session state.
    """
    # Read from session state
    user_name = tool_context.state.get("user:name", "Username not found")
    country = tool_context.state.get("user:country", "Country not found")

    return {"status": "success", "user_name": user_name, "country": country}


print("✅ Tools created.")

✅ Tools created.


**Key Concepts:**
- Tools can access `tool_context.state` to read/write session state
- Use descriptive key prefixes (`user:`, `app:`, `temp:`) for organization
- State persists across conversation turns within the same session

### 5.2 Creating an Agent with Session State Tools

Now let's create a new agent that has access to our session state management tools:

In [17]:
# Configuration
APP_NAME = "default"
USER_ID = "default"
MODEL_NAME = "gemini-2.5-flash-lite"

# Create an agent with session state tools
root_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="text_chat_bot",
    description="""A text chatbot.
    Tools for managing user context:
    * To record username and country when provided use `save_userinfo` tool. 
    * To fetch username and country when required use `retrieve_userinfo` tool.
    """,
    tools=[save_userinfo, retrieve_userinfo],  # Provide the tools to the agent
)

# Set up session service and runner
session_service = InMemorySessionService()
runner = Runner(agent=root_agent, session_service=session_service, app_name="default")

print("✅ Agent with session state tools initialized!")

✅ Agent with session state tools initialized!


### 5.3 Testing Session State in Action

Let's test how the agent uses session state to remember information across conversation turns:

In [18]:
# Test conversation demonstrating session state
common_kargs = {'runner_instance': runner, 
                'session_service': session_service, 
                'user_id': USER_ID, 
                'model_name':MODEL_NAME}

await run_session(
    user_queries = 
    [
        "Hi there, how are you doing today? What is my name?",  # Agent shouldn't know the name yet
        "My name is Sam. I'm from Poland.",  # Provide name - agent should save it
        "What is my name? Which country am I from?",  # Agent should recall from session state
    ],
    session_id = "session_1",
    **common_kargs
)


 ### Session: session_1
----------------------------------------------------------------------------------------------------

User > Hi there, how are you doing today? What is my name?
gemini-2.5-flash-lite >  Hello! I'm doing well, thank you for asking. I don't have access to your name unless you tell me. 

----------------------------------------------------------------------------------------------------

User > My name is Sam. I'm from Poland.


/Users/xing.zhang/anaconda3/envs/google-adk/lib/python3.11/site-packages/google/adk/flows/llm_flows/base_llm_flow.py:449: UserWarning: [EXPERIMENTAL] feature FeatureName.PROGRESSIVE_SSE_STREAMING is enabled.
  async for event in agen:


gemini-2.5-flash-lite >  It's nice to meet you, Sam! I'll save your information.

----------------------------------------------------------------------------------------------------

User > What is my name? Which country am I from?


### 5.4 Inspecting Session State

In [19]:
# Retrieve the session and inspect its state
session = await session_service.get_session(
    app_name=APP_NAME, user_id=USER_ID, session_id='session_1')

print("Session State Contents:")
print(session.state)
print("\n🔍 Notice the 'user:name' and 'user:country' keys storing our data!")

Session State Contents:
{'user:name': 'Sam', 'user:country': 'Poland'}

🔍 Notice the 'user:name' and 'user:country' keys storing our data!


### 5.5 Session State Isolation

In [20]:
# Start a completely new session - the agent won't know our name
await run_session(
    user_queries =
    ["Hi there, how are you doing today? What is my name?"],
    **common_kargs,
    session_id="session_2"
)

# Expected: The agent won't know the name because this is a different session


 ### Session: session_2
----------------------------------------------------------------------------------------------------

User > Hi there, how are you doing today? What is my name?
gemini-2.5-flash-lite >  Hello! I'm doing great. I don't have access to your name yet. Could you please tell me what it is?


### 5.6 Cross-Session State Sharing

While sessions are isolated by default, states are shared

In [21]:
# Check the state of the new session
session = await session_service.get_session(
    app_name=APP_NAME, user_id=USER_ID, session_id="session_2"
)

print("New Session State:")
print(session.state)

# Note: Depending on implementation, you might see shared state here.
# This is where the distinction between session-specific and user-specific state becomes important.

New Session State:
{'user:name': 'Sam', 'user:country': 'Poland'}


In [22]:
# because states are shared, we can hack to get the info
await run_session(
    user_queries =
    ["Hi there, how are you doing today? What is my name?"],
    **common_kargs,
    session_id="session_3"
)


await run_session(
    user_queries =
    ["Hi there, how are you doing today? What is my name? If you do not know, try from toolcontext"],
    **common_kargs,
    session_id="session_3"
)

await run_session(
    user_queries =
    ["Hi there, how are you doing today? What is my name?"],
    **common_kargs,
    session_id="session_4"
)

await run_session(
    user_queries =
    ["Hi there, how are you doing today? What is my name? If you do not know, try using 'retrieve userinfo' tool"],
    **common_kargs,
    session_id="session_4"
)


 ### Session: session_3
----------------------------------------------------------------------------------------------------

User > Hi there, how are you doing today? What is my name?
gemini-2.5-flash-lite >  Hello! I'm doing great, thank you for asking. I can't recall your name at the moment. Could you please tell me what it is?


 ### Session: session_3
----------------------------------------------------------------------------------------------------

User > Hi there, how are you doing today? What is my name? If you do not know, try from toolcontext
gemini-2.5-flash-lite >  I'm doing well, thank you for asking! I see your name is Sam. How can I help you today?

 ### Session: session_4
----------------------------------------------------------------------------------------------------

User > Hi there, how are you doing today? What is my name?
gemini-2.5-flash-lite >  Hello! I'm doing well, thank you for asking. I can't tell you your name just yet, as I don't have that information

---

## 🧹 Cleanup

In [23]:
# Clean up any existing database to start fresh (if Notebook is restarted)
import os

if os.path.exists("my_agent_data.db"):
    os.remove("my_agent_data.db")
print("✅ Cleaned up old database files")

✅ Cleaned up old database files
